In [1]:
#Pretty Display of Variables
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#%autoreload

# Import libraries

In [27]:
from plot_projections import plot_all_projections, plot_final_projections, plot_point_for_couple, plot_projection_up_down, plot_border
from prop_info import extreme_points, vect_blade, d_blade, center_prop
from get_segments import blade_alone, get_segments_points, get_planes
from major_axis import get_major_axis
#from projections import couple_all_planes, project_all_couples, projections_by_side, project_couple
from new_projections import assign_points, get_all_points_for_projections, interpolations, find_separation_plane, interpolate_points, model_func
from parameters import get_hub_points, get_hub_radius
from plot_param import plot_hub
from plot_prop import plot_pointcloud, plot_direction, plot_segments
from myMathFunction import least_squares
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

# Pre-processing

In [5]:
propeller_coords = pd.read_csv('aerostar_data.csv')
propeller_coords.shape

(151986, 3)

In [6]:
propeller_coords = propeller_coords.drop_duplicates(subset=None, keep='first', inplace=False)
propeller_coords = propeller_coords.reset_index(drop=True)

In [7]:
max_point, min_point, middle_point, highest_point, lowest_point = extreme_points(propeller_coords)

vect_length = vect_blade(max_point, min_point) 

dmiddle, dhighest, dlowest = d_blade(vect_length, middle_point, highest_point, lowest_point)

upper_blade, lower_blade = blade_alone(propeller_coords, vect_length, dmiddle)

vect_out, vect_side, hub_inner_radius = get_major_axis(propeller_coords, middle_point, vect_length)

# Projections

In [8]:
nb_seg = 3
resolution = 3

planes = get_planes(upper_blade, dmiddle, dhighest, vect_length, nb_seg)
segments = get_segments_points(upper_blade, planes, nb_seg)
#plot_segments(segments)

up, down = get_all_points_for_projections(planes, segments, nb_seg, resolution)

In [9]:
up1 = up[0]
dn1 = down[0]
print(up1.shape)

(322, 3)


In [10]:
# 1. Find border points
up_right_border, up_left_border, _, _, _ = extreme_points(up1)
dn_right_border, dn_left_border, _, _, _ = extreme_points(dn1)
#print("up_right {}\n".format(up_right))

In [11]:
# 2. Find separating plane
C_up = find_separation_plane(up1.values)
C_dn = find_separation_plane(dn1.values)
#print("C_up {}\n".format(C_up))

In [19]:
# 3. Assign point to side  (do it for both sides on both sides)
up_right_points, up_left_points = assign_points(C_up, up1)
dn_right_points, dn_left_points = assign_points(C_dn, dn1)
#print("right_points_up_shape {}\n".format(right_points_up.shape))

In [ ]:
# Add border points to fit
up_right_points_up.append(pd.DataFrame(up_right_border))
up_right_points_up.append(pd.DataFrame(up_left_border))
#print("right_points_up_shape {}\n".format(right_points_up.shape))

In [23]:
# 4. Interpolate points
up_right_popt = interpolate_points(up_right_points)
up_left_popt = interpolate_points(up_left_points)
dn_right_popt = interpolate_points(dn_right_points)
dn_left_popt = interpolate_points(dn_left_points)

def plot_interpolation(up1, popt):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    data = np.c_[up1.values[:,0], up1.values[:,1]]
    plt.plot(data, model_func(data, *popt), 'g--')
    plt.show

plot_interpolation(up_right_points, up_right_popt)

In [ ]:
# 5. Final projection

# Prendre 100 points d'un côté à lâutre de chaque courbe

# Relier 100 points en haut au 100 points en bas

# Garder les projections sur le plan



# Find parameters

    - Tip radius 
        Projection dans le sens de la longueur et ... ??
    - Blade twist 
        Project on planes as in projection
        Compute difference of angle between main directions of projections (PCA possible)

### Hub radius

In [ ]:
hub_points = get_hub_points(propeller_coords, dmiddle, vect_length)

hub_outer_radius, hub_inner_radius = get_hub_radius(hub_points, middle_point, vect_side)

hub_radius = hub_outer_radius[2] - middle_point[2]  #from center to exterior radius

In [ ]:
plot_hub(propeller_coords, hub_points, hub_outer_radius, hub_inner_radius)

### Chord length

In [ ]:
chord_length = np.linalg.norm(highest_point - middle_point) - hub_radius
chord_length

### Blade twist